In [ ]:
from scipy import sparse as sp
from matplotlib import pyplot as plt
import numpy as np

default_dtype = np.complex128

In [ ]:
# Copied from 05/lanczos.py
Id = sp.csr_matrix(np.eye(2), dtype=default_dtype)
Sx = sp.csr_matrix([[0., 1.], [1., 0.]], dtype=default_dtype)
Sz = sp.csr_matrix([[1., 0.], [0., -1.]], dtype=default_dtype)
Splus = sp.csr_matrix([[0., 1.], [0., 0.]], dtype=default_dtype)
Sminus = sp.csr_matrix([[0., 0.], [1., 0.]], dtype=default_dtype)


def singlesite_to_full(op, i, L):
    op_list = [Id]*L  # = [Id, Id, Id ...] with L entries
    op_list[i] = op
    full = op_list[0]
    for op_i in op_list[1:]:
        full = sp.kron(full, op_i, format="csr")
    return full


def gen_sx_list(L):
    return [singlesite_to_full(Sx, i, L) for i in range(L)]


def gen_sz_list(L):
    return [singlesite_to_full(Sz, i, L) for i in range(L)]


def _gen_hamiltonian(sx_list, sz_list, g, J=1.):
    L = len(sx_list)
    H = sp.csr_matrix((2**L, 2**L), dtype=default_dtype)
    for j in range(L):
        H += - g * sz_list[j]
        # open boundary
        if j:
            H += - J *( sx_list[j] * sx_list[(j-1)])
    return H

def gen_hamiltonian(L: int, g: float, J: float = 1.0) -> sp.csr_matrix:
    return _gen_hamiltonian(gen_sx_list(L), gen_sz_list(L), g, J)

In [ ]:
L = 14
J = 1
g = 1.5
H = gen_hamiltonian(L=L, g=g, J=J)
(E0,), psi_0 = sp.linalg.eigsh(H, k=1, which="SA")
psi_0 = psi_0.reshape(-1, 1)

# It's already normalized
assert np.allclose(np.dot(np.conj(psi_0).T, psi_0), 1)

print(f"{E0 = }")

In [ ]:
def compress(psi: np.ndarray, L: int, chi_max: int) -> list[np.ndarray]:
    """returns list of L 3D ndarrays"""
    res = []
    psi_n = psi.reshape(1, -1)
    for n in range(L):
        chi_n, dim_R_n = psi_n.shape
        psi_n = psi_n.reshape(2*chi_n, dim_R_n//2)

        M_n, lambda_n, psi_n_tilde = np.linalg.svd(psi_n, full_matrices=False)
                
        if lambda_n.size > chi_max:
            # Stolen from the exercise sheet
            keep = np.argsort(lambda_n)[:: -1][: chi_max ]  # indices to keep
            M_n = M_n[: , keep]                             # truncate matrix
            lambda_n = lambda_n[ keep ]                     # truncate lambdas
            psi_n_tilde = psi_n_tilde[ keep , :]            # truncate psi_tilde
        psi_n = lambda_n[:, np.newaxis] * psi_n_tilde[:, :]
        # End of stolen code
        
        chi_np1 = lambda_n.size

        res.append(M_n.reshape(chi_n, 2, chi_np1))
    return res


from functools import reduce

def compress_full_tensor(psi: np.ndarray, L: int, chi_max: int) -> np.ndarray:
    return reduce(
        lambda a, b: np.tensordot(a, b, axes=(-1, 0)), 
        compress(psi, L, chi_max),
        )


In [ ]:
# Check if we can recreate the original state from MPS
M = compress_full_tensor(psi_0, L, 2**(L//2))
print(f"{M.shape = }")
assert np.allclose(M.ravel(), psi_0.ravel())
del M

In [ ]:
# Find maximum compression ratio
# "Lossless" as defined by np.allclose
# Man, do while would be nice right now
def find_compression_ratio(psi: np.ndarray, L: int, chi_0: int = 20) -> None:
    M = compress_full_tensor(psi, L, 2**(L//2))
    chi = chi_0
    while np.allclose(M.ravel(), psi.ravel()):
        chi -= 1
        M = compress_full_tensor(psi, L, chi)
    
    if chi == chi_0:
        print("MPS representation did not correspond to original representation "
              f"with {chi_0 = }.")
        return

    # Found the first chi where we have compression losses
    chi += 1
    print(f"{chi = }")
    Ms_full_size = sum([M.size for M in compress(psi, L, 2**(L//2))])
    Ms_comp_size = sum([M.size for M in compress(psi, L, chi)])

    compression_ratio = Ms_full_size / Ms_comp_size
    print(f"{compression_ratio = :.2f}")

In [ ]:
print("Ground state: ")
find_compression_ratio(psi_0, L, 13)

In [ ]:
print("Random state: ")
psi_rnd = np.random.normal(size=(2**L)) + 1j * np.random.normal(size=(2**L))
psi_rnd /= np.dot(np.conj(psi_rnd).T, psi_rnd)
find_compression_ratio(psi_rnd, L, 2**(L//2))
del psi_rnd

In [ ]:
def overlap(MPS_a: list[np.ndarray], MPS_b: list[np.ndarray]) -> float:
    # Sum over alpha_0 and j_0
    res = np.tensordot(MPS_a[0], MPS_b[0].conj(), axes=((0, 1), (0, 1)))

    for Ma, Mb in zip(MPS_a[1:], MPS_b[1:]):
        # Sum over j_i
        T = np.tensordot(Ma, Mb.conj(), axes=(1, 1))
        # Sum over alpha_j both above and below
        res = np.tensordot(res, T, axes=((0, 1), (0, 2)))
    return res.item().real

In [ ]:
psi_exact = compress(psi_0, L, 2**(L//2) + 1)
psi_compr = compress(psi_0, L, 10)

In [ ]:
print(f"{overlap(psi_exact, psi_exact) = }")
print(f"{overlap(psi_compr, psi_compr) = }")
print(f"{overlap(psi_exact, psi_compr) = }")

In [ ]:
M_up = np.zeros((1, 2, 1))
M_up[0, 0, 0] = 1.
MPS_all_up = [M_up.copy() for _ in range(L)]
print(f"{overlap(psi_exact, MPS_all_up) = }")
